In [ ]:
from modelo.problema import Problema
from modelo.reporte import Reporte
import pandas as pd
from datetime import datetime, timedelta
from modelo.reporte import Reporte
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
file = '0_model_23oct-rev.xlsm'

problema = Problema(excel_file_path=file)

problema.generar_sets()

problema.generar_parameters()

problema.generar_vars()

problema.generar_target()

problema.gen_constrains()

problema.solve(tlimit=120)

In [ ]:
reporte = Reporte(problema=problema)

In [ ]:
planta_df = reporte.obtener_fact_inventario_planta()

In [ ]:
filtros = ['empresa', 'planta', 'ingrediente', 'item']

In [ ]:
names = [datetime.strptime(x, '%Y-%m-%d')  for x in planta_df.drop(columns=['Previo'] + filtros).columns]
names = [names[0]-timedelta(days=1)] + names
names = [x.strftime(format='%b-%d') for x in names]

In [ ]:
ingrediente = 'maiz'
planta = 'envigado'

df = planta_df[(planta_df['ingrediente']==ingrediente)&(planta_df['planta']==planta)]

In [ ]:
def dibujar(df:pd.DataFrame):
    # Values of each group
    llegadas_directas_kg = list(df[df['item']=='llegadas_directas_kg'].drop(columns=filtros).iloc[0])
    llegadas_por_bodega_kg = list(df[df['item']=='llegadas_por_bodega_kg'].drop(columns=filtros).iloc[0])
    inventario_al_cierre = list(df[df['item']=='inventario_al_cierre_kg'].drop(columns=filtros).iloc[0])
    transitos_a_bodega = list(df[df['item']=='transitos_kg'].drop(columns=filtros).iloc[0])
    consumo_proyectado = list(df[df['item']=='consumo_kg'].drop(columns=filtros).iloc[0])
    safety_stock = list(df[df['item']=='safety_stock'].drop(columns=filtros).iloc[0])
    
    fig, ax = plt.subplots(figsize=(12, 4))

    fig.suptitle(f'Inventarios de {ingrediente} en {planta}', fontsize=14)

    ax.set_xlabel('Fecha')
    ax.set_ylabel('Kg')

    ax.plot(names, inventario_al_cierre, label='Inventario al cierre')
    ax.plot(names, consumo_proyectado, label='Consumo Proyectado')
    ax.plot(names, safety_stock, color='red', markers='-', label='Inventario de seguridad')
    ax.bar(names, llegadas_directas_kg, color='blue', width=1, label='Llegadas directas')
    ax.bar(names, llegadas_por_bodega_kg, color='red', width=1, label='Llegadas indirectas')
    ax.bar(names, transitos_a_bodega, color='red', width=1, label='Llegadas programadas')


    fig.autofmt_xdate()

    plt.legend()
    
    # Show graphic
    return fig